In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from iowalicor.data.preprocessing import slugify, get_dataset_slice

In [3]:
dataset_file = "../data/raw/Iowa_Liquor_Sales.csv"
dataset = get_dataset_slice(dataset_file, random_state=35, shuffle=True)
dataset

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S15031600166,10/09/2013,2501.0,Hy-vee #2 / Ames,640 LINCOLNWAY,AMES,50010,"640 LINCOLNWAY\nAMES 50010\n(42.022848, -93.61...",85.0,Story,...,34122.0,Crystal Head Vodka,6.0,750.0,$26.00,$39.00,2.0,$78.00,1.5,0.40
1,S07718900007,09/13/2012,2190.0,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,"1460 2ND AVE\nDES MOINES 50314\n(41.60566, -93...",77.0,Polk,...,16676.0,Basil Hayden 8YR,6.0,750.0,$20.47,$30.70,6.0,$184.20,4.5,1.19
2,S17733000061,03/05/2014,4180.0,Smokin' Joe's #10 Tobacco and Liquor,480 7TH AVE,MARION,52302,"480 7TH AVE\nMARION 52302\n(42.033308, -91.60546)",57.0,Linn,...,34001.0,Absolut Swedish Vodka 80 Prf Mini,10.0,600.0,$7.92,$11.88,1.0,$11.88,0.6,0.16
3,S19318900003,06/03/2014,2578.0,Hy-Vee / Charles City,901 KELLY ST,CHARLES CITY,50616,"901 KELLY ST\nCHARLES CITY 50616\n(43.066994, ...",34.0,Floyd,...,11777.0,Black Velvet,12.0,1000.0,$6.63,$9.94,36.0,$357.84,36.0,9.51
4,S06492900071,07/10/2012,2506.0,Hy-Vee #1044 / Burlington,3140 AGENCY,BURLINGTON,52601,"3140 AGENCY\nBURLINGTON 52601\n(40.814689, -91...",29.0,Des Moines,...,35318.0,Barton Vodka,6.0,1750.0,$6.87,$10.30,6.0,$61.80,10.5,2.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121812,S23712800116,01/28/2015,4167.0,"Iowa Street Market, Inc.",1256 IOWA ST,DUBUQUE,52001,"1256 IOWA ST\nDUBUQUE 52001\n(42.504958, -90.6...",31.0,Dubuque,...,43418.0,Captain Morgan Tattoo,12.0,750.0,$8.75,$13.13,2.0,$26.26,1.5,0.40
121813,S21841300006,10/22/2014,3879.0,Target Store T-1800 / Sioux City,5775 SUNNYBROOK DR,SIOUX CITY,51106,5775 SUNNYBROOK DR\nSIOUX CITY 51106\n(42.4489...,97.0,Woodbury,...,1400.0,Crown Royal w/2 Glasses,6.0,750.0,$14.99,$22.49,30.0,$674.70,22.5,5.94
121814,S27864700021,09/14/2015,2630.0,Hy-Vee Drugstore #2 / WDM,1010 60TH ST,WEST DES MOINES,50266,1010 60TH ST\nWEST DES MOINES 50266\n(41.58497...,77.0,Polk,...,37348.0,Phillips Vodka,6.0,1750.0,$7.60,$11.40,12.0,$136.80,21.0,5.55
121815,S27858500002,09/14/2015,4402.0,KUM & GO #80 / RIVERSIDE,1178 ENTERPRISE DR,RIVERSIDE,52327,1178 ENTERPRISE DR\nRIVERSIDE 52327\n,92.0,Washington,...,75214.0,Kinky Blue,12.0,750.0,$10.00,$15.00,2.0,$30.00,1.5,0.40


In [4]:
dataset.drop(columns=["Item Description"], inplace=True)
location = np.array([
    (splitted[0], splitted[1].strip(")"))
    if (len(splitted := str(a).split("\n(")) > 1)
    else (str(a), None) for a in dataset["Store Location"]
])
dataset.drop(columns=["Store Location"], inplace=True)
dataset["Store Coordinates"] = location[:, 1]
dataset = dataset[dataset["Store Coordinates"].notnull()]
latitude, longitude = zip(*[[float(el.split(",")[0]), float(el.split(",")[1])] for el in dataset["Store Coordinates"]])
dataset.drop(columns=["Store Coordinates"])
dataset["Latitude"] = latitude
dataset["Longitude"] = longitude
dataset.County.fillna("no_county", inplace=True)
dataset.County = [slugify(count) for count in dataset.County]
dataset.City = [slugify(str(cit)) for cit in dataset.City]
dataset["Store Name"] = [slugify(str(name).split("/")[0]) for name in dataset["Store Name"]]
dataset["Vendor Name"] = [slugify(str(vname)) for vname in dataset["Vendor Name"]]
dataset["Address"] = [slugify(str(addr)) for addr in dataset["Address"]]
date_list = [datetime.strptime(str(date), "%m/%d/%Y") for date in dataset.Date]
dataset.drop(columns=["Date"], inplace=True)
dataset["Year"] = [date.year for date in date_list]
dataset["Month"] = [date.month for date in date_list]
dataset["Day"] = [date.day for date in date_list]
dataset["Sale (Dollars)"] = dataset["Sale (Dollars)"].map(lambda el: float(el.strip("$")))


/tmp/ipykernel_4435/4157791280.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Latitude"] = latitude
/tmp/ipykernel_4435/4157791280.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Longitude"] = longitude
/home/aizen/.cache/pypoetry/virtualenvs/iowalicor-e8HhPf_A-py3.8/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [5]:
dummied_dataset = pd.get_dummies(dataset, columns=[
    "County",
    "City",
    "Category Name",
    "Store Name",
    "Vendor Name",
    "Address"
])

In [6]:
for col in dummied_dataset.columns:
    print(col)

Invoice/Item Number
Store Number
Zip Code
County Number
Category
Vendor Number
Item Number
Pack
Bottle Volume (ml)
State Bottle Cost
State Bottle Retail
Bottles Sold
Sale (Dollars)
Volume Sold (Liters)
Volume Sold (Gallons)
Store Coordinates
Latitude
Longitude
Year
Month
Day
County_adair
County_adams
County_allamakee
County_appanoose
County_audubon
County_benton
County_black_hawk
County_boone
County_bremer
County_buchanan
County_buena_vista
County_butler
County_calhoun
County_carroll
County_cass
County_cedar
County_cerro_gordo
County_cherokee
County_chickasaw
County_clarke
County_clay
County_clayton
County_clinton
County_crawford
County_dallas
County_davis
County_decatur
County_delaware
County_des_moines
County_dickinson
County_dubuque
County_emmet
County_fayette
County_floyd
County_franklin
County_fremont
County_greene
County_grundy
County_guthrie
County_hamilton
County_hancock
County_hardin
County_harrison
County_henry
County_howard
County_humboldt
County_ida
County_iowa
County_jacks

In [7]:
for col in dummied_dataset.columns:
    print(col, {type(el) for el in dummied_dataset[col]})

Invoice/Item Number {<class 'str'>}
Store Number {<class 'float'>}
Zip Code {<class 'str'>}
County Number {<class 'float'>}
Category {<class 'float'>}
Vendor Number {<class 'str'>}
Item Number {<class 'float'>}
Pack {<class 'float'>}
Bottle Volume (ml) {<class 'float'>}
State Bottle Cost {<class 'str'>}
State Bottle Retail {<class 'str'>}
Bottles Sold {<class 'float'>}
Sale (Dollars) {<class 'float'>}
Volume Sold (Liters) {<class 'float'>}
Volume Sold (Gallons) {<class 'float'>}
Store Coordinates {<class 'str'>}
Latitude {<class 'float'>}
Longitude {<class 'float'>}
Year {<class 'int'>}
Month {<class 'int'>}
Day {<class 'int'>}
County_adair {<class 'int'>}
County_adams {<class 'int'>}
County_allamakee {<class 'int'>}
County_appanoose {<class 'int'>}
County_audubon {<class 'int'>}
County_benton {<class 'int'>}
County_black_hawk {<class 'int'>}
County_boone {<class 'int'>}
County_bremer {<class 'int'>}
County_buchanan {<class 'int'>}
County_buena_vista {<class 'int'>}
County_butler {<cla

In [ ]:
dummied_dataset["Store Number"]